<a href="https://colab.research.google.com/github/monikagithub1234/calculator-project/blob/main/resume_classification_shortlisting_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Install Required Libraries**

In [ ]:
pip install pandas numpy scikit-learn tensorflow

**Load and Prepare Dataset**

In [ ]:
import pandas as pd

# Load dataset
df = pd.read_csv("/content/UpdatedResumeDataSet.csv")  # Update path

# Show sample
print(df.head())


       Category                                             Resume
0  Data Science  Skills * Programming Languages: Python (pandas...
1  Data Science  Education Details \r\nMay 2013 to May 2017 B.E...
2  Data Science  Areas of Interest Deep Learning, Control Syste...
3  Data Science  Skills â¢ R â¢ Python â¢ SAP HANA â¢ Table...
4  Data Science  Education Details \r\n MCA   YMCAUST,  Faridab...


**Preprocess Text Data**

In [ ]:
import re
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd # Make sure pandas is imported if this is a separate cell

# Clean text
def clean_text(text):
    text = re.sub(r'[^\w\s]', '', text.lower())
    return text

df['Cleaned_Resume'] = df['Resume'].apply(clean_text)

# TF-IDF Vectorization
tfidf = TfidfVectorizer(max_features=3000)
X = tfidf.fit_transform(df['Cleaned_Resume']).toarray()

# Labels
# Check the column names to find the correct one
print(df.columns)

# Once you know the correct column name, replace 'Shortlisted' below
# For example, if the column is named 'shortlisted_status', change the line to:
# y = df['shortlisted_status']
# Assuming the correct column name is 'Category' based on the df variable output
y = df['Category']

Index(['Category', 'Resume', 'Cleaned_Resume'], dtype='object')


**Train/Test Split**

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:
# Build ANN Model (Using Keras)
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# ANN model
model = Sequential()
model.add(Dense(128, activation='relu', input_shape=(X_train.shape[1],)))
model.add(Dense(64, activation='relu'))
model.add(Dense(1, activation='sigmoid'))  # Binary output

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 128)            │       384,128 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 392,449 (1.50 MB)

 Trainable params: 392,449 (1.50 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
# Train the Model
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
import numpy as np # Import numpy if not already imported

# Convert string labels to integers
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.transform(y_test) # Use transform on test set

# One-hot encode the integer labels
y_train_categorical = to_categorical(y_train_encoded)
y_test_categorical = to_categorical(y_test_encoded)

# Update the output layer of the model to match the number of classes
num_classes = y_train_categorical.shape[1]
# Rebuild the model with the correct output layer if it was already defined
# Note: It's better to define the model *after* determining num_classes if possible
# or modify the existing model if it's the only way in your notebook structure.
# Since the model is defined in a previous cell, we'll redefine and recompile it here
# for simplicity in this fix, but ideally, the model definition should happen
# after determining `num_classes`.

# Redefine and recompile the model
model = Sequential()
model.add(Dense(128, activation='relu', input_shape=(X_train.shape[1],)))
model.add(Dense(64, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))  # Use softmax for multiclass

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

# Now train the model with the one-hot encoded labels
history = model.fit(X_train, y_train_categorical, epochs=10, batch_size=32, validation_split=0.2)

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_3 (Dense)                 │ (None, 128)            │       384,128 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 25)             │         1,625 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 394,009 (1.50 MB)

 Trainable params: 394,009 (1.50 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 2s 25ms/step - accuracy: 0.3337 - loss: 3.1729 - val_accuracy: 0.5584 - val_loss: 2.9914
Epoch 2/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - accuracy: 0.7094 - loss: 2.7982 - val_accuracy: 0.5974 - val_loss: 2.4520
Epoch 3/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.7279 - loss: 2.0585 - val_accuracy: 0.7468 - val_loss: 1.7083
Epoch 4/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.8829 - loss: 1.2044 - val_accuracy: 0.9091 - val_loss: 1.0104
Epoch 5/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.9872 - loss: 0.5980 - val_accuracy: 0.9870 - val_loss: 0.5447
Epoch 6/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 1.0000 - loss: 0.2872 - val_accuracy: 1.0000 - val_loss: 0.2933
Epoch 7/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 1.0000 - loss: 0.1489 - val_accuracy: 1.0000 - val_loss: 0.1780
Epoch 8/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 1.0000 - loss: 0.0863 - val_accuracy: 1.0000 - v

In [ ]:

# Evaluate the model using the one-hot encoded test labels
loss, accuracy = model.evaluate(X_test, y_test_categorical) # Use y_test_categorical instead of y_test
print(f"Test Accuracy: {accuracy*100:.2f}%")

7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.9973 - loss: 0.0530 
Test Accuracy: 99.48%


In [ ]:
#  Make Predictions
sample_resume = ["I have experience in data science and Python"]
sample_clean = [clean_text(sample_resume[0])]
sample_vec = tfidf.transform(sample_clean).toarray()
prediction = model.predict(sample_vec)
print("Shortlisted" if prediction[0][0] > 0.5 else "Not Shortlisted")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/step
Not Shortlisted


In [ ]:
#  Save Model

model.save("resume_shortlist_model.h5")

**✅ Modified Output with Shortlist + Confidence Score**

In [ ]:
import pandas as pd
import re
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# Step 1: Load the dataset
# Ensure this path is correct for the dataset you intend to use
df = pd.read_csv("/content/Resume.csv")

# Step 2: Text Cleaning Function
def clean_text(text):
    return re.sub(r'[^\w\s]', '', text.lower())

df['Cleaned_Resume'] = df['Resume'].apply(clean_text)

# Step 3: Vectorize text using TF-IDF
tfidf = TfidfVectorizer(max_features=3000)
X = tfidf.fit_transform(df['Cleaned_Resume']).toarray()

# Step 4: Encode target labels
le = LabelEncoder()
# Change 'Category' to 'Shortlisted' to match the column in the loaded DataFrame
y = le.fit_transform(df['Shortlisted'])

# Optional: Print category mapping
print("Label mapping:", dict(zip(le.classes_, le.transform(le.classes_))))

# Step 5: Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Step 6: Build ANN model
model = Sequential()
model.add(Dense(128, activation='relu', input_shape=(X_train.shape[1],)))
model.add(Dense(64, activation='relu'))
# The number of output nodes should match the number of unique classes
# Since we are using LabelEncoder on the 'Shortlisted' column which likely has two classes (Shortlisted/Not Shortlisted or similar)
# the number of classes will be 2. Softmax activation is appropriate for multi-class classification.
# Using len(le.classes_) ensures the output layer size matches the number of unique labels.
model.add(Dense(len(le.classes_), activation='softmax'))

# Use 'sparse_categorical_crossentropy' as the loss function when labels are integers (not one-hot encoded)
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Step 7: Train model
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2)

# Step 8: Evaluate
loss, accuracy = model.evaluate(X_test, y_test)
print(f"\nTest Accuracy: {accuracy*100:.2f}%")

# Step 9: Predict on new input
sample_resume = ["Skilled in Python, data analysis, and machine learning"]
sample_clean = [clean_text(sample_resume[0])]
sample_vec = tfidf.transform(sample_clean).toarray()

pred = model.predict(sample_vec)
# Inverse transform the predicted integer label back to its original string representation
predicted_label = le.inverse_transform([pred.argmax()])[0]

print("\nPredicted Category:", predicted_label)
if predicted_label == 1:
    print("Shortlisted ✅")
else:
    print("Not Shortlisted ❌")

Label mapping: {np.int64(0): np.int64(0), np.int64(1): np.int64(1)}
Epoch 1/10


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step - accuracy: 0.5000 - loss: 0.6726 - val_accuracy: 0.0000e+00 - val_loss: 0.6988
Epoch 2/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 114ms/step - accuracy: 0.8333 - loss: 0.6490 - val_accuracy: 0.0000e+00 - val_loss: 0.6987
Epoch 3/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 127ms/step - accuracy: 1.0000 - loss: 0.6265 - val_accuracy: 0.5000 - val_loss: 0.6990
Epoch 4/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 129ms/step - accuracy: 1.0000 - loss: 0.6048 - val_accuracy: 0.5000 - val_loss: 0.6995
Epoch 5/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 116ms/step - accuracy: 1.0000 - loss: 0.5835 - val_accuracy: 0.5000 - val_loss: 0.7002
Epoch 6/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 121ms/step - accuracy: 1.0000 - loss: 0.5632 - val_accuracy: 0.5000 - val_loss: 0.7013
Epoch 7/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 112ms/step - accuracy: 1.0000 - loss: 0.5438 - val_accuracy: 0.5000 - val_loss: 0.7026
Epoch 8/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 139ms/step - accuracy: 1.0000 - loss: 0.5246 - val_accuracy: 0.5000 - val_loss: 0.703

In [ ]:
# Add a Skill Match Score (Bonus Idea)
required_skills = ['python', 'sql', 'machine learning', 'data science']

def skill_match_score(resume_text):
    resume_text = clean_text(resume_text)
    matched = [skill for skill in required_skills if skill in resume_text]
    return len(matched), matched

count, matched = skill_match_score(sample_resume[0])
print(f"Skills matched: {count} ➤ {matched}")


Skills matched: 2 ➤ ['python', 'machine learning']


In [30]:
# my resume
from IPython import get_ipython
from IPython.display import display
# %% [markdown]
# **Install Required Libraries**
# %%
!pip install pandas numpy scikit-learn tensorflow
# %% [markdown]
# **Load and Prepare Dataset**
# %%
import pandas as pd

# Load dataset
# Using the specified file path /content/Resume.csv
try:
    # *** IMPORTANT ***
    # Ensure that the file at this path (/content/Resume.csv) actually
    # contains the columns 'Resume' and 'Category'. If it doesn't,
    # you will get a KeyError later.
    df = pd.read_csv("/content/Resume.csv")
    print("Successfully loaded /content/Resume.csv")
    # Show sample
    print(df.head())
    print("\nColumns:", df.columns.tolist()) # Print column names to verify
except FileNotFoundError:
    print("Error: /content/Resume.csv not found. Please check the file path.")
    # You might want to exit or handle this error appropriately
except Exception as e:
    print(f"An error occurred while loading the dataset from /content/Resume.csv: {e}")
    # Handle other potential loading errors


# %% [markdown]
# **Preprocess Text Data**
# %%
import re
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
# pandas is already imported in the previous cell, but good practice to re-import if this were a standalone cell
# import pandas as pd
from sklearn.preprocessing import LabelEncoder # Import LabelEncoder here as it's used in the next block
import numpy as np # Import numpy as it's used later in prediction

# Clean text function
def clean_text(text):
    # Handle potential NaN values in the text column
    if isinstance(text, str):
        text = re.sub(r'[^\w\s]', '', text.lower())
        return text
    else:
        return "" # Return an empty string for non-string inputs

# Ensure df is loaded and 'Resume' column exists before processing
if 'df' in locals() and 'Resume' in df.columns:
    df['Cleaned_Resume'] = df['Resume'].apply(clean_text)
    print("\n'Cleaned_Resume' column created.")
    print(df[['Resume', 'Cleaned_Resume']].head()) # Show sample of cleaned data

    # TF-IDF Vectorization
    tfidf = TfidfVectorizer(max_features=3000)

    # Ensure 'Cleaned_Resume' column exists before vectorizing
    if 'Cleaned_Resume' in df.columns:
        X = tfidf.fit_transform(df['Cleaned_Resume']).toarray()
        print(f"\nTF-IDF vectorization complete. Shape of X: {X.shape}")
    else:
        print("Error: 'Cleaned_Resume' column not found after cleaning. Cannot perform TF-IDF vectorization.")

    # Labels
    # Assuming 'Category' is the correct column name from the dataset you are using
    if 'Category' in df.columns:
        y = df['Category']
        print(f"\n'Category' column loaded. Number of samples: {len(y)}")

        # Convert string labels to integers using LabelEncoder
        label_encoder = LabelEncoder()
        y_encoded = label_encoder.fit_transform(y)
        print("Labels encoded.")

        # Print label mapping to understand the integer representation of categories
        print("Label mapping:", dict(zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_))))
        print(f"Shape of y_encoded: {y_encoded.shape}")

    else:
        print("Error: 'Category' column not found in the DataFrame. Cannot encode labels.")

else:
    print("Error: DataFrame not loaded or 'Resume' column missing. Skipping preprocessing.")


# %% [markdown]
# **Train/Test Split**
# %%
# Ensure X and y_encoded were successfully created in the previous step
if 'X' in locals() and 'y_encoded' in locals():
    # Use the encoded labels for the train/test split
    X_train, X_test, y_train_encoded, y_test_encoded = train_test_split(X, y_encoded, test_size=0.2, random_state=42)
    print(f"\nTrain/Test split complete. Shapes:")
    print(f"  X_train: {X_train.shape}")
    print(f"  X_test: {X_test.shape}")
    print(f"  y_train_encoded: {y_train_encoded.shape}")
    print(f"  y_test_encoded: {y_test_encoded.shape}")
else:
    print("Error: X or y_encoded not defined. Skipping train/test split.")


# %%
# Build ANN Model (Using Keras)
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.utils import to_categorical
# numpy is already imported in the previous cell, but good practice to re-import if standalone
# import numpy as np

# Ensure label_encoder was successfully created
if 'label_encoder' in locals():
    # Determine the number of classes from the encoded labels
    num_classes = len(label_encoder.classes_)
    print(f"\nNumber of classes: {num_classes}")

    # Ensure y_train_encoded and y_test_encoded were successfully created
    if 'y_train_encoded' in locals() and 'y_test_encoded' in locals():
        # One-hot encode the integer labels
        y_train_categorical = to_categorical(y_train_encoded, num_classes=num_classes)
        y_test_categorical = to_categorical(y_test_encoded, num_classes=num_classes)
        print("Labels one-hot encoded.")
        print(f"  y_train_categorical: {y_train_categorical.shape}")
        print(f"  y_test_categorical: {y_test_categorical.shape}")

        # Ensure X_train is defined for input shape
        if 'X_train' in locals():
            # ANN model
            model = Sequential()
            model.add(Dense(128, activation='relu', input_shape=(X_train.shape[1],)))
            model.add(Dense(64, activation='relu'))
            # The output layer should have 'num_classes' nodes with 'softmax' activation for multi-class classification
            model.add(Dense(num_classes, activation='softmax'))

            # Use 'categorical_crossentropy' for loss when using one-hot encoded labels
            model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
            print("\nANN Model built and compiled.")
            model.summary()
        else:
            print("Error: X_train not defined. Cannot build model.")
    else:
        print("Error: Encoded labels not defined. Cannot build model.")
else:
    print("Error: label_encoder not defined. Cannot determine number of classes.")


# %%
# Train the Model
# Ensure the model and training data are defined
if 'model' in locals() and 'X_train' in locals() and 'y_train_categorical' in locals():
    print("\nStarting model training...")
    history = model.fit(X_train, y_train_categorical, epochs=10, batch_size=32, validation_split=0.2)
    print("Model training finished.")
else:
    print("Error: Model or training data not defined. Cannot train model.")

# %%
# Evaluate the model using the one-hot encoded test labels
# Ensure the model and test data are defined
if 'model' in locals() and 'X_test' in locals() and 'y_test_categorical' in locals():
    print("\nEvaluating model on test data...")
    loss, accuracy = model.evaluate(X_test, y_test_categorical)
    print(f"Test Loss: {loss:.4f}")
    print(f"Test Accuracy: {accuracy*100:.2f}%")
else:
    print("Error: Model or test data not defined. Cannot evaluate model.")


# %%
#  Make Predictions
# Ensure clean_text, tfidf, model, and label_encoder are defined
if 'clean_text' in locals() and 'tfidf' in locals() and 'model' in locals() and 'label_encoder' in locals():
    sample_resume = ["I have experience in data science and Python"]
    print(f"\nSample resume for prediction: '{sample_resume[0]}'")

    sample_clean = [clean_text(sample_resume[0])]
    print(f"Cleaned sample resume: '{sample_clean[0]}'")

    # Ensure tfidf was fit on the training data and can transform new data
    try:
        sample_vec = tfidf.transform(sample_clean).toarray()
        print(f"Sample resume vectorized. Shape: {sample_vec.shape}")

        # Make prediction
        prediction_probs = model.predict(sample_vec)
        print(f"Prediction probabilities (for all categories): {prediction_probs[0]}") # Print probabilities for the first sample

        # Get the predicted class index (the one with the highest probability)
        predicted_class_index = np.argmax(prediction_probs, axis=1)[0]
        print(f"Predicted class index: {predicted_class_index}")

        # Inverse transform the predicted class index back to the original category name
        predicted_category = label_encoder.inverse_transform([predicted_class_index])[0]

        # Print the predicted category
        print(f"\nPredicted Category: {predicted_category}")

        # --- ADD SHORTLISTING LOGIC HERE ---
        # Define which categories are considered 'Shortlisted'
        # *** IMPORTANT: EDIT THIS LIST ***
        # Replace these example categories with the actual category names
        # from your 'label_encoder.classes_' that you want to shortlist.
        shortlisted_categories = ['Data Science', 'Machine Learning', 'Web Development', 'Python Developer'] # <-- **EDIT THIS LIST**

        # Check if the predicted category is in the list of shortlisted categories
        if predicted_category in shortlisted_categories:
            print("Shortlisted ✅")
            # Optional: You could also get the confidence score for the predicted category
            confidence_score = prediction_probs[0][predicted_class_index]
            print(f"Confidence Score for Predicted Category: {confidence_score:.4f}")
        else:
            print("Not Shortlisted ❌")

        # --- END SHORTLISTING LOGIC ---


    except Exception as e:
        print(f"An error occurred during prediction: {e}")

else:
    print("Error: Required components for prediction (clean_text, tfidf, model, label_encoder) not defined.")

# %%
#  Save Model
# Ensure the model is defined
if 'model' in locals():
    # Ensure directory exists or save to a path where you have write permissions
    try:
        model.save("resume_classification_model.h5")
        print("\nModel saved successfully as resume_classification_model.h5")
    except Exception as e:
        print(f"\nError saving model: {e}")
        print("Please ensure you have write permissions in the current directory.")
else:
    print("\nError: Model not defined. Cannot save model.")


# %% [markdown]
# **Add a Skill Match Score (Bonus Idea)**
# %%
# Add a Skill Match Score (Bonus Idea)
# Ensure clean_text is defined (it's defined in the main preprocessing section)
if 'clean_text' in locals():
    required_skills = ['python', 'sql', 'machine learning', 'data science']

    def skill_match_score(resume_text):
        # Handle potential non-string input
        if isinstance(resume_text, str):
            resume_text = clean_text(resume_text)
            matched = [skill for skill in required_skills if skill in resume_text]
            return len(matched), matched
        else:
            return 0, []


    # Ensure sample_resume is defined (it is defined in the 'Make Predictions' cell)
    if 'sample_resume' in locals():
        print("\nCalculating skill match for sample resume:")
        count, matched = skill_match_score(sample_resume[0])
        print(f"Skills matched: {count} ➤ {matched}")
    else:
        print("\nError: sample_resume not defined. Cannot calculate skill match score.")
else:
     print("\nError: clean_text function not defined. Cannot calculate skill match score.")

Successfully loaded /content/Resume.csv
  Unnamed: 0        Unnamed: 1  \
0        NaN               NaN   
1        NaN               NaN   
2        NaN               NaN   
3       Year       Degree/Exam   
4       2026  B.Tech, CSE(CSM)   

                                  NAGAM MONIKA PRIYA       Unnamed: 3  
0                  Email: monikapriyanagam@gmail.com              NaN  
1                                  Number:9989527325              NaN  
2                                          EDUCATION              NaN  
3                                          Institute  CGPA/Percentage  
4  Kakinada institute of engineering and technolo...            7.31%  

Columns: ['Unnamed: 0', 'Unnamed: 1', 'NAGAM MONIKA PRIYA', 'Unnamed: 3']
Error: DataFrame not loaded or 'Resume' column missing. Skipping preprocessing.

Train/Test split complete. Shapes:
  X_train: (769, 3000)
  X_test: (193, 3000)
  y_train_encoded: (769,)
  y_test_encoded: (193,)

Number of classes: 25
Labels one-ho

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_7"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_21 (Dense)                │ (None, 128)            │       384,128 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_22 (Dense)                │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_23 (Dense)                │ (None, 25)             │         1,625 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 394,009 (1.50 MB)

 Trainable params: 394,009 (1.50 MB)

 Non-trainable params: 0 (0.00 B)


Starting model training...
Epoch 1/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 6s 67ms/step - accuracy: 0.3150 - loss: 3.1780 - val_accuracy: 0.5649 - val_loss: 3.0045
Epoch 2/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 2s 28ms/step - accuracy: 0.7241 - loss: 2.8129 - val_accuracy: 0.6234 - val_loss: 2.4684
Epoch 3/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - accuracy: 0.7641 - loss: 2.0267 - val_accuracy: 0.7143 - val_loss: 1.7099
Epoch 4/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.8897 - loss: 1.1776 - val_accuracy: 0.8961 - val_loss: 1.0413
Epoch 5/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.9733 - loss: 0.6245 - val_accuracy: 1.0000 - val_loss: 0.5662
Epoch 6/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 1.0000 - loss: 0.3266 - val_accuracy: 1.0000 - val_loss: 0.3054
Epoch 7/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 1.0000 - loss: 0.1397 - val_accuracy: 1.0000 - val_loss: 0.1882
Epoch 8/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 1.0000 - loss: 0.087


Model saved successfully as resume_classification_model.h5

Calculating skill match for sample resume:
Skills matched: 2 ➤ ['python', 'data science']
